## Layer Data
The purpose of this notebook is to gather the details about each layer for each snow pit with no tree canopy in the Grand Mesa. 

### Parameters
* Depth 
    * Each depth for each measurement layer is rounded to the nearest multiple of 10 centimeters
* Density
    * Density rounded to each 10 
* Temperature 
    * Temperature taken at each layer, assuming temperature is the temperature is the same for the entire layer, given a single measurement
* Height
    * Max of each density for each snow pit
* Grain Size 
    * Conversions of data made:
        * "< 1 mm" = 0.5    
        * "1-2 mm" = 1.5
        * "2-4 mm" = 3
        * "4-6 mm " = 5
        * "> 6 mm" = 6.5
        * "[]" = NULL
    * If there are multiple grain sizes in a single layer, we are taking the median

In [ ]:
--snow pit density
SELECT DISTINCT depth, bottom_depth, value , date 
FROM public.layers WHERE pit_id = 'COGM8C22_20200131' and type = 'density' ORDER BY depth;


In [ ]:
--snow pit temp
SELECT DISTINCT depth, value , date 
FROM public.layers WHERE pit_id = 'COGM8C22_20200131' and type = 'temperature' ORDER BY depth;
 

In [ ]:
"
;With Height as (
    --snow pit height
    SELECT DISTINCT 
        pit_id
        , date
        , type
        , max(depth) as height
    FROM public.layers 
    WHERE type = 'density'
    GROUP BY 
        pit_id
        , date
        , type
), 
Density as (
    --snow pit density
    SELECT DISTINCT 
        pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
        , depth as actual_depth
        , bottom_depth
        , value as density
    FROM public.layers 
    WHERE type = 'density'
), 
Temp as (
    --snow pit temp
    SELECT pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
        , depth as actual_depth
        , value as temperature
    FROM public.layers WHERE type = 'temperature'
), 
Grain as (
    --snow pit grain size
    SELECT pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
           , depth as actual_depth

        , value as grain_size
    FROM public.layers WHERE type = 'grain_size'
)

SELECT DISTINCT 
    S.site_name
    , L.pit_id
    , S.latitude
    , S.longitude
    , L.date
    , H.height
    , T.depth
    , D.density
    , T.temperature
    , G.grain_size
FROM public.sites S 
    INNER JOIN public.layers L on L.pit_id = S.pit_id and L.date = S.date
    LEFT JOIN Height H on H.pit_id = L.pit_id and H.date = L.date
    LEFT JOIN Density D on D.pit_id = L.pit_id and D.date = L.date and D.depth = ROUND(CAST(L.depth as numeric), -1)
    LEFT JOIN Temp T on T.pit_id = L.pit_id and T.date = L.date and T.depth =  ROUND(CAST(L.depth as numeric), -1)
    LEFT JOIN Grain G on G.pit_id = L.pit_id and G.date = L.date and G.depth = ROUND(CAST(L.depth as numeric), -1)
WHERE  S.site_name = 'Grand Mesa' and S.tree_canopy = 'No Trees' 
ORDER BY L.pit_id, T.depth
"

In [ ]:
;With Height as (
    --snow pit height
    SELECT DISTINCT 
        pit_id
        , date
        , type
        , max(depth) as height
    FROM public.layers 
    WHERE type = 'density'
    GROUP BY 
        pit_id
        , date
        , type
), 
Density as (
    --snow pit density
    SELECT DISTINCT 
        pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
        , depth as actual_depth
        , bottom_depth
        , value as density
    FROM public.layers 
    WHERE type = 'density'
), 
Temp as (
    --snow pit temp
    SELECT pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
        , depth as actual_depth
        , value as temperature
    FROM public.layers WHERE type = 'temperature'
), 
Grain as (
    --snow pit grain size
    SELECT pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
           , depth as actual_depth

        , value as grain_size
    FROM public.layers WHERE type = 'grain_size'
)

SELECT DISTINCT 
    S.site_name
    , L.pit_id
    , S.latitude
    , S.longitude
    , L.date
    , H.height
    , T.depth
    , D.density
    , T.temperature
    , G.grain_size
FROM public.sites S 
    INNER JOIN public.layers L on L.pit_id = S.pit_id and L.date = S.date
    LEFT JOIN Height H on H.pit_id = L.pit_id and H.date = L.date
    LEFT JOIN Density D on D.pit_id = L.pit_id and D.date = L.date and D.depth = ROUND(CAST(L.depth as numeric), -1)
    LEFT JOIN Temp T on T.pit_id = L.pit_id and T.date = L.date and T.depth =  ROUND(CAST(L.depth as numeric), -1)
    LEFT JOIN Grain G on G.pit_id = L.pit_id and G.date = L.date and G.depth = ROUND(CAST(L.depth as numeric), -1)
WHERE  S.site_name = 'Grand Mesa' and S.tree_canopy = 'No Trees' 
ORDER BY L.pit_id, T.depth



/*

--layer data
SELECT DISTINCT type FROM public.layers limit(100)

--snow pit density
SELECT DISTINCT depth, bottom_depth, value , date
FROM public.layers WHERE pit_id = 'COGM8C22_20200131' and type = 'density' ORDER BY depth;


--snow pit temp
SELECT DISTINCT depth, value , date, * 
FROM public.layers WHERE pit_id = 'COGM8C22_20200131' and type = 'temperature' ORDER BY depth;

--site data # 137 pits
SELECT distinct pit_id FROM public.sites where site_name = 'Grand Mesa' and tree_canopy = 'No Trees'

SELECT Latitude, Longitude FROM public.sites where site_name = 'Grand Mesa' and tree_canopy = 'No Trees'

SELECT DISTINCT value
FROM public.layers WHERE type = 'grain_size';



*/

In [9]:
# Import the connection function from the snowexsql library
from snowexsql.db import get_db

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)


# Form a typical SQL query and use python to populate the table name
qry = "SELECT DISTINCT site_id FROM sites"

# Then we execute the sql command and collect the results
results = engine.execute(qry)

In [10]:
results

In [24]:
qry = '''
;With Height as (
    --snow pit height
    SELECT DISTINCT 
        pit_id
        , date
        , type
        , max(depth) as height
    FROM public.layers 
    WHERE type = 'density'
    GROUP BY 
        pit_id
        , date
        , type
), 
Density as (
    --snow pit density
    SELECT DISTINCT 
        pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
        , bottom_depth
        , value as density
    FROM public.layers 
    WHERE type = 'density'
), 
Temp as (
    --snow pit temp
    SELECT pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
        , value as temperature
    FROM public.layers 
    WHERE type = 'temperature'
), 
Grain as (
    --snow pit grain size
    SELECT pit_id
        , date
        , type
        , ROUND(CAST(depth as numeric), -1) as depth
        , CASE 
            WHEN value = '< 1 mm' then 0.5  
            WHEN value = '1-2 mm' then 1.5
            WHEN value = '2-4 mm' then 3
            WHEN value = '2-4mm' then 3
            WHEN value = '4-6 mm' then 5
            WHEN value = '4-6 mm ' then 5
            WHEN value = '> 6 mm' then 6.5
            else NULL 
        END as grain_size
        , value as original_grain_size
    FROM public.layers 
    WHERE type = 'grain_size'
)
SELECT DISTINCT 
    S.site_name
    , L.pit_id
    , S.latitude
    , S.longitude
    , L.date
    , H.height
    , T.depth
    , D.density
    , T.temperature
    , G.grain_size
    , G.original_grain_size
FROM public.sites S 
    INNER JOIN public.layers L on L.pit_id = S.pit_id and L.date = S.date
    LEFT JOIN Height H on H.pit_id = L.pit_id and H.date = L.date
    LEFT JOIN Density D on D.pit_id = L.pit_id and D.date = L.date and D.depth = ROUND(CAST(L.depth as numeric), -1)
    LEFT JOIN Temp T on T.pit_id = L.pit_id and T.date = L.date and T.depth =  ROUND(CAST(L.depth as numeric), -1)
    LEFT JOIN Grain G on G.pit_id = L.pit_id and G.date = L.date and G.depth = ROUND(CAST(L.depth as numeric), -1)
WHERE S.site_name = 'Grand Mesa' and S.tree_canopy = 'No Trees' 
ORDER BY L.pit_id, T.depth
'''

In [25]:
# Import the connection function from the snowexsql library
from snowexsql.db import get_db

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)


In [41]:
import pandas as pd

df = pd.read_sql(qry, engine)
df

,site_name,pit_id,latitude,longitude,date,height,depth,density,temperature,grain_size,original_grain_size
0,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,0.0,None,-1.1,NaN,None
1,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,10.0,None,-2.2,NaN,None
2,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,20.0,284.0,-3.3,3.0,2-4 mm
3,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,30.0,315.5,-4.3,1.5,1-2 mm
4,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,40.0,271.0,-5.5,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
1736,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,90.0,413.3333333333333,-0.2,NaN,None
1737,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,100.0,358.0,-0.2,0.5,< 1 mm
1738,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,100.0,358.0,-0.2,NaN,None
1739,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,110.0,332.5,-0.1,0.5,< 1 mm


In [54]:
df.iloc[:, :-2]

,site_name,pit_id,latitude,longitude,date,height,depth,density,temperature
0,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,0.0,None,-1.1
1,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,10.0,None,-2.2
2,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,20.0,284.0,-3.3
3,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,30.0,315.5,-4.3
4,Grand Mesa,COGM1C14_20200131,39.047216,-108.198415,2020-01-31,81.0,40.0,271.0,-5.5
...,...,...,...,...,...,...,...,...,...
1736,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,90.0,413.3333333333333,-0.2
1737,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,100.0,358.0,-0.2
1738,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,100.0,358.0,-0.2
1739,Grand Mesa,COGMWO_20200409_1615,39.033980,-108.213900,2020-04-09,113.0,110.0,332.5,-0.1


In [52]:
df_medians = df.groupby(['pit_id', 'depth'])['grain_size'].median().reset_index()
df_medians.head()

,pit_id,depth,grain_size
0,COGM1C14_20200131,0.0,NaN
1,COGM1C14_20200131,10.0,NaN
2,COGM1C14_20200131,20.0,3.0
3,COGM1C14_20200131,30.0,1.5
4,COGM1C14_20200131,40.0,NaN


In [55]:
df_all = df_medians.merge(df.iloc[:, :-2], on=('pit_id', 'depth'), how='left')

In [59]:
df_all = df_all.drop_duplicates()

In [61]:
df_all

,pit_id,depth,grain_size,site_name,latitude,longitude,date,height,density,temperature
0,COGM1C14_20200131,0.0,NaN,Grand Mesa,39.047216,-108.198415,2020-01-31,81.0,None,-1.1
1,COGM1C14_20200131,10.0,NaN,Grand Mesa,39.047216,-108.198415,2020-01-31,81.0,None,-2.2
2,COGM1C14_20200131,20.0,3.0,Grand Mesa,39.047216,-108.198415,2020-01-31,81.0,284.0,-3.3
3,COGM1C14_20200131,30.0,1.5,Grand Mesa,39.047216,-108.198415,2020-01-31,81.0,315.5,-4.3
4,COGM1C14_20200131,40.0,NaN,Grand Mesa,39.047216,-108.198415,2020-01-31,81.0,271.0,-5.5
...,...,...,...,...,...,...,...,...,...,...
1715,COGMWO_20200409_1615,80.0,NaN,Grand Mesa,39.033980,-108.213900,2020-04-09,113.0,387.3333333333333,-0.2
1716,COGMWO_20200409_1615,90.0,0.5,Grand Mesa,39.033980,-108.213900,2020-04-09,113.0,413.3333333333333,-0.2
1718,COGMWO_20200409_1615,100.0,0.5,Grand Mesa,39.033980,-108.213900,2020-04-09,113.0,358.0,-0.2
1720,COGMWO_20200409_1615,110.0,0.5,Grand Mesa,39.033980,-108.213900,2020-04-09,113.0,332.5,-0.1
